In [1]:
import rouge

from tqdm import tqdm
from utils import read_lines

In [2]:
avg_evaluator = rouge.Rouge(metrics=['rouge-n', 'rouge-l', 'rouge-w'],
                            max_n=4,
                            limit_length=True,
                            length_limit=200,
                            length_limit_type='words',
                            apply_avg=True,
                            apply_best=False,
                            alpha=0.5, # Default F1_score
                            weight_factor=1.2,
                            stemming=True)

#### Read File

In [9]:
BEAM_SIZE = 4

In [10]:
train_preds_path = 'preds/cnndm_train_bm4_all_merged.hypo'
train_target_path = '/home/ml/cadencao/Two-Steps-Summarization/datasets/cnn_dm/fairseq_files/train.target'

In [11]:
train_preds = read_lines(train_preds_path)
train_target = read_lines(train_target_path)

In [12]:
train_preds = [train_preds[i::BEAM_SIZE] for i in range(BEAM_SIZE)]

In [13]:
assert len(train_preds[0]) == len(train_target), "{}/{}".format(len(train_preds[0]), len(train_target))

#### Select Best Beam

In [14]:
index, best_predictions = 0, []
for t in tqdm(train_target):
    best_rouge_1, best_pred = -1.0, None
    for s in range(BEAM_SIZE):
        p = train_preds[s][index]
        score = avg_evaluator.get_scores([p], [t])
        if score['rouge-1']['f'] > best_rouge_1:
            best_rouge_1 = score['rouge-1']['f']
            best_pred = p
    best_predictions.append(best_pred)
    index += 1

 20%|██        | 58851/287227 [29:58<1:51:17, 34.20it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|██████████| 287227/287227 [2:50:03<00:00, 28.15it/s]  


In [15]:
assert len(best_predictions) == len(train_target)

In [16]:
with open('preds/cnndm_train_best_beam.hypo', 'w') as f:
    for p in best_predictions:
        f.write(p + '\n')